In [2]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
sys.path.append('../smodels')
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from getRefXSecs import getXSecFor
pd.options.mode.chained_assignment = None #Disable copy warnings

In [7]:
### Add reference cross-section ratio for C1C1,C1N2:
pids = [[-1000024,1000024],[1000023,1000024],[-1000024,1000023]]
ratio = []
xsecRefSum = 0.0
xsecSum = 0.0
row = {'mass.1000023' : 2158.35734, 'mass.1000024' : 2158.35734 }
for pidpair in pids:
    mass1 = row['mass.%i'%abs(pidpair[0])] 
    mass2 = row['mass.%i'%abs(pidpair[1])] 
    mass = (mass1+mass2)/2.
    xsecRef = max(0,1000*getXSecFor(pidpair[0],pidpair[1],mass,13.0,'wino'))
    xsecRefSum += xsecRef
    print(pidpair,xsecRef)
print(xsecRefSum)

[-1000024, 1000024] 0.00017155624881599755
[1000023, 1000024] 0.0003730965046480004
[-1000024, 1000023] 0.0001795718045839995
0.0007242245580479974


In [5]:
slhaFolder = '../EWino/data/slha_scanRandom/'
slhaFile = 'ew_ahtejtpl.slha'
slhaData = pyslha.readSLHAFile(os.path.join(slhaFolder,slhaFile))

In [8]:
mC1 = slhaData.blocks['MASS'][1000024]
mN2 = slhaData.blocks['MASS'][1000023]
mN1 = slhaData.blocks['MASS'][1000022]
M1 = slhaData.blocks['EXTPAR'][1]
M2 = slhaData.blocks['EXTPAR'][2]
mu = slhaData.blocks['EXTPAR'][23]
print('mC1 = %1.3f, mN2 = %1.3f, mLSP = %1.3f' %(mC1,mN2,mN1))
print('M1 = %1.3f, M2 = %1.3f, mu = %1.3f' %(M1,M2,mu))

print('\nC1 decays:')
print('ctau (m) = %1.2e' %(1.967e-16/slhaData.decays[1000024].totalwidth))
for dec in slhaData.decays[1000024].decays:
    if dec.br < 1e-3: continue
    print(dec)
    
print('\nN2 decays:')
print('ctau (m) = %1.2e' %(1.967e-16/slhaData.decays[1000023].totalwidth))
for dec in slhaData.decays[1000023].decays:
    if dec.br < 1e-3: continue
    print(dec)

xsecC1N2 = 0.
xsecC1N2 += max([x.value for x in slhaData.xsections[(2212,2212,1000023,1000024)].xsecs if x.sqrts == 13000.])
xsecC1N2 += max([x.value for x in slhaData.xsections[(2212,2212,-1000024,1000023)].xsecs if x.sqrts == 13000.])
      
xsecRef = max(0,getXSecFor(1000023,1000024,(mC1+mN2)/2.,13.0,'wino'))
xsecRef += max(0,getXSecFor(-1000024,1000023,(mC1+mN2)/2.,13.0,'wino'))

print('\nC1pmN2 xsec: %1.2e pb' %xsecC1N2)        
print('C1pmN2 ref xsec: %1.2e pb' %xsecRef)

mC1 = 2158.505, mN2 = 2158.357, mLSP = 1884.185
M1 = 1849.063, M2 = 2019.940, mu = 2785.804

C1 decays:
ctau (m) = 2.62e-14
 1 [24, 1000022]

N2 decays:
ctau (m) = 2.36e-14
 1 [25, 1000022]
 0.0049 [23, 1000022]

C1pmN2 xsec: 1.94e-06 pb
C1pmN2 ref xsec: 5.50e-07 pb
